In [10]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import json

import cv2
import numpy as np

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        super().__init__()
        self.img_dir = img_dir
        self.data_file = data_file
        with open(data_file, "r") as f:
            self.data = json.load(f)
            
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        item = self.data[index]
        img = cv2.imread(f"{self.img_dir}/{item["id"]}.png")
        h,w = img.shape[:2]
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transform(img)
        kps = np.array(item["kps"]).flatten()
        kps = kps.astype(np.float32)
        
        kps[::2] *= 224.0 / w
        kps[1::2] *= 224.0 / h
        
        return img, kps
        
train_data = KeypointsDataset("/home/rwang2000/datasets/tennis_court_det_dataset/data/images", "/home/rwang2000/datasets/tennis_court_det_dataset/data/data_train.json")
val_data = KeypointsDataset("/home/rwang2000/datasets/tennis_court_det_dataset/data/images", "/home/rwang2000/datasets/tennis_court_det_dataset/data/data_val.json")

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
train_loader = DataLoader(val_data, batch_size=32, shuffle=False)


In [14]:
model = models.resnet50(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 14*2)

model = model.to(device)

/home/rwang2000/miniforge3/envs/py312/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rwang2000/miniforge3/envs/py312/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/rwang2000/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:02<00:00, 43.0MB/s]


In [15]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [16]:
epochs = 20

for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)
        
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()
        
        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

Epoch 0, iter 0, loss: 15072.9560546875
Epoch 0, iter 10, loss: 14415.513671875
Epoch 0, iter 20, loss: 14720.3232421875
Epoch 0, iter 30, loss: 13711.9833984375
Epoch 0, iter 40, loss: 13414.369140625
Epoch 0, iter 50, loss: 13088.8427734375
Epoch 0, iter 60, loss: 12831.3583984375
Epoch 1, iter 0, loss: 12582.587890625
Epoch 1, iter 10, loss: 11815.91015625
Epoch 1, iter 20, loss: 12035.8740234375
Epoch 1, iter 30, loss: 11052.1591796875
Epoch 1, iter 40, loss: 10747.4677734375
Epoch 1, iter 50, loss: 10446.4375
Epoch 1, iter 60, loss: 10201.2099609375
Epoch 2, iter 0, loss: 9974.0693359375
Epoch 2, iter 10, loss: 9277.947265625
Epoch 2, iter 20, loss: 9503.0859375
Epoch 2, iter 30, loss: 8640.4794921875
Epoch 2, iter 40, loss: 8390.650390625
Epoch 2, iter 50, loss: 8158.22119140625
Epoch 2, iter 60, loss: 7951.1455078125
Epoch 3, iter 0, loss: 7766.0458984375
Epoch 3, iter 10, loss: 7147.20361328125
Epoch 3, iter 20, loss: 7373.9208984375
Epoch 3, iter 30, loss: 6620.896484375
Epoch

In [17]:
torch.save(model.state_dict(), "models/tennis_court_keypoints.pth")